# EDA - Variables Ordinales et Catégorielles

**Objectif** : Analyser les 96 variables ordinales et 70 variables catégorielles du dataset PISA pour guider le preprocessing.

**Données** :
- Dataset : `data/X_train.csv`
- Classification : `data/classification_variables.xlsx`
- Glossaire : `data/Glossaire.xlsx`
- Documentation : `preprocessing/glossaire_analysis.md`

**Note** : On utilise **Polars** pour des performances optimales sur les gros datasets.

## 1. Setup et imports

In [24]:
import polars as pl
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy.stats import spearmanr
from scipy.stats import chi2_contingency
import warnings

# Pour lire les fichiers Excel (Polars ne supporte pas directement Excel)
import pandas as pd

warnings.filterwarnings('ignore')
sns.set_style('whitegrid')
plt.rcParams['figure.figsize'] = (12, 6)

print("✅ Imports réussis")
print(f"Polars version: {pl.__version__}")

✅ Imports réussis
Polars version: 1.35.2


## 2. Chargement des données

### 2.1 Charger échantillon de X_train.csv

In [25]:
# Charger un échantillon avec Polars (beaucoup plus rapide que pandas)
SAMPLE_SIZE = 50000

print(f"Chargement de {SAMPLE_SIZE} lignes de X_train.csv avec Polars...")
df = pl.read_csv('../data/X_train.csv', n_rows=SAMPLE_SIZE)

print(f"\n✅ Dataset chargé : {df.shape[0]:,} lignes, {df.shape[1]} colonnes")
print(f"Mémoire estimée : {df.estimated_size('mb'):.2f} MB")

Chargement de 50000 lignes de X_train.csv avec Polars...

✅ Dataset chargé : 50,000 lignes, 307 colonnes
Mémoire estimée : 102.50 MB


In [26]:
# Aperçu des données
df.head()

,Year,CNT,CNTRYID,CNTSCHID,CNTSTUID,CYC,NatCen,STRATUM,SUBNATIO,OECD,ADMINMODE,LANGTEST_QQQ,LANGTEST_COG,LANGTEST_PAQ,Option_CT,Option_FL,Option_ICTQ,Option_WBQ,Option_PQ,Option_TQ,Option_UH,ST001D01T,ST003D02T,ST003D03T,ST004D01T,EFFORT1,EFFORT2,OCOD1,OCOD2,OCOD3,AGE,GRADE,ISCEDP,IMMIG,COBN_S,MISSSC,…,math_q4_total_timing,math_q5_total_timing,math_q6_total_timing,math_q7_total_timing,math_q8_total_timing,math_q9_total_timing,math_q10_total_timing,math_q11_total_timing,math_q12_total_timing,math_q13_total_timing,math_q14_total_timing,math_q15_total_timing,math_q16_total_timing,math_q17_total_timing,math_q18_total_timing,math_q19_total_timing,math_q20_total_timing,math_q21_total_timing,science_q1_total_timing,science_q2_total_timing,science_q3_total_timing,science_q4_total_timing,science_q5_total_timing,science_q6_total_timing,science_q7_total_timing,science_q8_total_timing,science_q9_total_timing,science_q10_total_timing,science_q11_total_timing,science_q12_total_timing,science_q13_total_timing,science_q14_total_timing,science_q15_total_timing,science_q16_total_timing,science_q17_total_timing,science_q18_total_timing,science_q19_total_timing
i64,i64,str,f64,f64,f64,str,i64,str,i64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,…,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,str,str,str,str,str,str,str,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,str,str,str,str,str,str,str
384002,2022,"""NLD""",528.0,5.2800132e7,5.2801144e7,"""08MS""",52800,"""NLD06""",5280000,1.0,2.0,322.0,322.0,null,1.0,1.0,0.0,1.0,0.0,0.0,1.0,10.0,2.0,2007.0,1.0,8.0,8.0,3256.0,2212.0,9999.0,15.75,0.0,244.0,2.0,52800.0,0.0,…,110641.333333,96368.333333,71813.5,98875.0,63689.4,37232.0,88274.666667,null,null,null,null,null,null,null,null,null,null,null,84299.333333,90156.0,66612.0,73547.6,68912.0,56623.25,104190.5,132623.25,105785.666667,null,null,null,null,null,null,null,null,null,null
1118072,2018,"""QAZ""",31.0,3.100106e6,3.100424e6,"""07MS""",3100,"""QAZ0101""",310000,0.0,2.0,803.0,803.0,null,null,null,null,null,null,null,null,10.0,11.0,2002.0,2.0,7.0,6.0,2330.0,9313.0,0.0,15.42,0.0,null,null,3100.0,null,…,116718.0,61003.0,34117.0,56901.667969,76615.0,21500.333984,27087.0,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null
845454,2018,"""FRA""",250.0,2.500001e7,2.5005207e7,"""07MS""",25000,"""FRA0101""",2500000,1.0,2.0,493.0,493.0,null,null,null,null,null,null,null,null,10.0,5.0,2002.0,1.0,null,null,3323.0,3322.0,7512.0,16.0,0.0,null,null,25000.0,null,…,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,81443.664062,190243.40625,36257.0,148073.5,2121.249756,90540.335938,57954.0,84633.0,45332.0,87686.5,13164.75,1187.199951,null,null,null,null,null,null,null
1728613,2015,"""QES""",971.0,9.710024e7,9.7127584e7,"""06MS""",72400,"""ESP1633""",7241600,2.0,2.0,156.0,156.0,null,null,1.0,1.0,null,1.0,1.0,0.0,10.0,3.0,1999.0,2.0,null,null,null,null,null,16.17,null,null,1.0,null,null,…,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,62224.667969,83289.664062,91510.75,100649.5,82378.5,74348.25,91230.664062,49789.0,37855.0,null,null,null,null,null,null,null,null,null,null
1083243,2018,"""PHL""",608.0,6.0800071e7,6.0802698e7,"""07MS""",60800,"""PHL0011""",6080000,0.0,2.0,313.0,313.0,null,null,null,null,null,null,null,null,8.0,9.0,2002.0,1.0,10.0,10.0,9701.0,7132.0,2161.0,15.5,-1.0,null,null,60800.0,null,…,82282.5,121010.5,191303.0,77024.335938,197619.671875,64659.667969,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null


In [27]:
# Info générale (schema = types de colonnes en Polars)
print("Schema du dataset :")
print(df.schema)

print(f"\nNombre total de colonnes : {len(df.columns)}")
print(f"\nPremières colonnes : {df.columns[:20]}")

Schema du dataset :
Schema({'': Int64, 'Year': Int64, 'CNT': String, 'CNTRYID': Float64, 'CNTSCHID': Float64, 'CNTSTUID': Float64, 'CYC': String, 'NatCen': Int64, 'STRATUM': String, 'SUBNATIO': Int64, 'OECD': Float64, 'ADMINMODE': Float64, 'LANGTEST_QQQ': Float64, 'LANGTEST_COG': Float64, 'LANGTEST_PAQ': Float64, 'Option_CT': Float64, 'Option_FL': Float64, 'Option_ICTQ': Float64, 'Option_WBQ': Float64, 'Option_PQ': Float64, 'Option_TQ': Float64, 'Option_UH': Float64, 'ST001D01T': Float64, 'ST003D02T': Float64, 'ST003D03T': Float64, 'ST004D01T': Float64, 'EFFORT1': Float64, 'EFFORT2': Float64, 'OCOD1': Float64, 'OCOD2': Float64, 'OCOD3': Float64, 'AGE': Float64, 'GRADE': Float64, 'ISCEDP': Float64, 'IMMIG': Float64, 'COBN_S': Float64, 'MISSSC': Float64, 'MATHEASE': Float64, 'WB153': Float64, 'ST253': Float64, 'ST311': Float64, 'WB165': Float64, 'IC171': Float64, 'ST038': Float64, 'IC180': Float64, 'ST255': Float64, 'ST350': Float64, 'ST034': Float64, 'FL162': Float64, 'PA195': Float64, 

### 2.2 Charger les dictionnaires de référence du Glossaire

Note: Pour lire Excel, on utilise pandas puis on convertit en Polars

In [28]:
# Charger toutes les feuilles du Glossaire (avec pandas car Excel)
glossaire_path = '../data/Glossaire.xlsx'

print("Chargement des feuilles du Glossaire...")
glossaire_sheets = pd.ExcelFile(glossaire_path).sheet_names
print(f"Feuilles disponibles : {glossaire_sheets}")

# Charger la feuille principale (General) et convertir en Polars
glossaire_general_pd = pd.read_excel(glossaire_path, sheet_name='General')
glossaire_general = pl.from_pandas(glossaire_general_pd)

print(f"\n✅ Glossaire General : {glossaire_general.shape}")
glossaire_general.head()

Chargement des feuilles du Glossaire...
Feuilles disponibles : ['General', 'CNT', 'CNTRYID', 'STRATUM', 'ISCEDP', 'OCOD']

✅ Glossaire General : (308, 3)


question_code,description,thematic domain
str,str,str
"""Year""","""Year of the test""","""General"""
"""CNT""","""Country code 3-character""","""General"""
"""CNTRYID""","""Country Identifier""","""General"""
"""CNTSCHID""","""International School ID""","""General"""
"""CNTSTUID""","""International Student ID""","""General"""


In [29]:
# Charger les dictionnaires de référence
reference_dicts = {}

for sheet in ['CNT', 'CNTRYID', 'STRATUM', 'ISCEDP', 'OCOD']:
    if sheet in glossaire_sheets:
        # Charger avec pandas puis convertir en Polars
        temp_pd = pd.read_excel(glossaire_path, sheet_name=sheet)
        reference_dicts[sheet] = pl.from_pandas(temp_pd)
        print(f"{sheet}: {reference_dicts[sheet].shape[0]} entrées")

print(f"\n✅ Dictionnaires chargés : {list(reference_dicts.keys())}")

CNT: 80 entrées
CNTRYID: 80 entrées
STRATUM: 1316 entrées
ISCEDP: 59 entrées
OCOD: 620 entrées

✅ Dictionnaires chargés : ['CNT', 'CNTRYID', 'STRATUM', 'ISCEDP', 'OCOD']


## 3. Identification des variables ordinales et catégorielles

La classification est donnée dans `data/classification_variables.xlsx`

Selon `glossaire_analysis.md` :
- 96 variables ordinales (31.2%)
- 70 variables catégorielles (22.7%)
- 135 variables numériques (43.8%)
- 7 variables de groupement (2.3%)

In [30]:
# Charger la classification des variables
classification_path = '../data/classification_variables.xlsx'

print("Chargement de la classification des variables...")
classification_pd = pd.read_excel(classification_path)
classification = pl.from_pandas(classification_pd)

print(f"\n✅ Classification chargée : {classification.shape}")
print(f"\nColonnes disponibles : {classification.columns}")
classification.head(10)

Chargement de la classification des variables...

✅ Classification chargée : (135, 4)

Colonnes disponibles : ['Variables Numériques', 'Variables Ordinales', 'Variables Catégorielles', 'Variables de Groupement']


Variables Numériques,Variables Ordinales,Variables Catégorielles,Variables de Groupement
str,str,str,str
"""AGE""","""ISCEDP""","""CYC""","""Year"""
"""COBN_S""","""ST253""","""NatCen""","""CNT"""
"""WB165""","""ST311""","""SUBNATIO""","""CNTRYID"""
"""FL169""","""IC171""","""OECD""","""CNTSCHID"""
"""WB166""","""ST038""","""ADMINMODE""","""CNTSTUID"""
"""FL164""","""IC180""","""LANGTEST_QQQ""","""STRATUM"""
"""ST355""","""ST255""","""LANGTEST_COG""","""BOOKID"""
"""IC177""","""ST034""","""LANGTEST_PAQ""",null
"""IC183""","""PA195""","""Option_CT""",null


In [31]:
# Extraire les listes de variables depuis les colonnes
# Chaque colonne contient une liste de codes de variables

# Variables ordinales : toutes les valeurs non-nulles de la colonne "Variables Ordinales"
ordinal_vars = classification['Variables Ordinales'].drop_nulls().to_list()

# Variables catégorielles : toutes les valeurs non-nulles de la colonne "Variables Catégorielles"
categorical_vars = classification['Variables Catégorielles'].drop_nulls().to_list()

# Variables numériques : toutes les valeurs non-nulles de la colonne "Variables Numériques"
numeric_vars = classification['Variables Numériques'].drop_nulls().to_list()

# Variables de groupement : toutes les valeurs non-nulles de la colonne "Variables de Groupement"
grouping_vars = classification['Variables de Groupement'].drop_nulls().to_list()

print(f"✅ Variables ordinales : {len(ordinal_vars)}")
print(f"✅ Variables catégorielles : {len(categorical_vars)}")
print(f"✅ Variables numériques : {len(numeric_vars)}")
print(f"✅ Variables de groupement : {len(grouping_vars)}")

print(f"\n📋 Exemples de variables ordinales : {ordinal_vars[:10]}")
print(f"📋 Exemples de variables catégorielles : {categorical_vars[:10]}")

✅ Variables ordinales : 96
✅ Variables catégorielles : 70
✅ Variables numériques : 135
✅ Variables de groupement : 7

📋 Exemples de variables ordinales : ['ISCEDP', 'ST253', 'ST311', 'IC171', 'ST038', 'IC180', 'ST255', 'ST034', 'PA195', 'ST230']
📋 Exemples de variables catégorielles : ['CYC', 'NatCen', 'SUBNATIO', 'OECD', 'ADMINMODE', 'LANGTEST_QQQ', 'LANGTEST_COG', 'LANGTEST_PAQ', 'Option_CT', 'Option_FL']


In [32]:
# Vérifier quelles variables de la classification sont présentes dans le dataset
dataset_cols = set(df.columns)
ordinal_in_dataset = [v for v in ordinal_vars if v in dataset_cols]
categorical_in_dataset = [v for v in categorical_vars if v in dataset_cols]

print(f"\n📊 PRÉSENCE DANS LE DATASET")
print(f"Variables ordinales présentes : {len(ordinal_in_dataset)} / {len(ordinal_vars)}")
print(f"Variables catégorielles présentes : {len(categorical_in_dataset)} / {len(categorical_vars)}")

# Variables manquantes
ordinal_missing = [v for v in ordinal_vars if v not in dataset_cols]
categorical_missing = [v for v in categorical_vars if v not in dataset_cols]

if ordinal_missing:
    print(f"\n⚠️ Variables ordinales manquantes : {len(ordinal_missing)}")
    print(f"Exemples : {ordinal_missing[:5]}")

if categorical_missing:
    print(f"\n⚠️ Variables catégorielles manquantes : {len(categorical_missing)}")
    print(f"Exemples : {categorical_missing[:5]}")

# Mettre à jour les listes pour ne garder que les variables présentes
ordinal_vars = ordinal_in_dataset
categorical_vars = categorical_in_dataset

print(f"\n✅ Listes mises à jour avec variables présentes dans le dataset")
print(f"Ordinales : {len(ordinal_vars)}")
print(f"Catégorielles : {len(categorical_vars)}")


📊 PRÉSENCE DANS LE DATASET
Variables ordinales présentes : 95 / 96
Variables catégorielles présentes : 70 / 70

⚠️ Variables ordinales manquantes : 1
Exemples : ['MathScore']

✅ Listes mises à jour avec variables présentes dans le dataset
Ordinales : 95
Catégorielles : 70


C'est normal, c'est le label.

## 4. Analyse des variables ordinales

Pour chaque variable ordinale :
- Cardinalité (nombre de valeurs uniques)
- % de valeurs manquantes
- Type d'échelle (Likert, fréquence, quantité)
- Valeurs aberrantes ou codes spéciaux (-99, 97, 98, 99)

In [33]:
# Fonction pour analyser une variable ordinale avec Polars
def analyze_ordinal_variable(df: pl.DataFrame, var_name: str) -> dict:
    """Analyse une variable ordinale avec Polars."""
    if var_name not in df.columns:
        return None
    
    col = df[var_name]
    
    # Statistiques de base
    n_unique = col.n_unique()
    missing_count = col.null_count()
    missing_pct = (missing_count / df.height) * 100
    
    # Valeurs uniques (limitées à 20) - convertir en strings pour éviter problèmes de types mixtes
    unique_vals_raw = col.drop_nulls().unique().to_list()
    unique_vals = str(sorted(unique_vals_raw)[:20]) if unique_vals_raw else "[]"
    
    # Value counts (top 10) - convertir en string pour éviter problèmes avec None/null
    value_counts = df.group_by(var_name).agg(pl.count().alias('count')).sort('count', descending=True).head(10)
    value_counts_dict = str({str(row[0]): row[1] for row in value_counts.iter_rows()})
    
    # Codes spéciaux (si numérique)
    if col.dtype in [pl.Int8, pl.Int16, pl.Int32, pl.Int64, pl.Float32, pl.Float64]:
        has_negative = int((col < 0).sum())
        has_high_codes = int(((col >= 97) & (col <= 99)).sum())
    else:
        has_negative = 0
        has_high_codes = 0
    
    analysis = {
        'variable': var_name,
        'n_unique': int(n_unique),
        'missing_count': int(missing_count),
        'missing_pct': float(missing_pct),
        'dtype': str(col.dtype),
        'unique_values': unique_vals,
        'value_counts': value_counts_dict,
        'has_negative_codes': has_negative,
        'has_high_codes': has_high_codes
    }
    
    return analysis

print("✅ Fonction analyze_ordinal_variable définie")

✅ Fonction analyze_ordinal_variable définie


In [34]:
# Analyser toutes les variables ordinales
if 'ordinal_vars' in locals() and len(ordinal_vars) > 0:
    ordinal_analysis = []
    
    print(f"Analyse de {len(ordinal_vars)} variables ordinales...")
    for var in ordinal_vars:
        result = analyze_ordinal_variable(df, var)
        if result:
            ordinal_analysis.append(result)
    
    # Convertir en DataFrame Polars
    ordinal_df = pl.DataFrame(ordinal_analysis)
    print(f"\n✅ Analyse de {len(ordinal_df)} variables ordinales terminée")
    ordinal_df.head(10)
else:
    print("⚠️ Liste des variables ordinales non disponible")

Analyse de 95 variables ordinales...

✅ Analyse de 95 variables ordinales terminée


In [35]:
# Statistiques sur les variables ordinales
if 'ordinal_df' in locals():
    print("\n📊 STATISTIQUES VARIABLES ORDINALES")
    print("=" * 60)
    print(f"Nombre total : {ordinal_df.height}")
    print(f"\nMissing values :")
    print(f"  - Moyenne : {ordinal_df['missing_pct'].mean():.2f}%")
    print(f"  - Médiane : {ordinal_df['missing_pct'].median():.2f}%")
    print(f"  - Max : {ordinal_df['missing_pct'].max():.2f}%")
    print(f"\nCardinalité :")
    print(f"  - Moyenne : {ordinal_df['n_unique'].mean():.1f}")
    print(f"  - Médiane : {ordinal_df['n_unique'].median():.1f}")
    print(f"  - Max : {ordinal_df['n_unique'].max()}")
    print(f"\nCodes spéciaux :")
    print(f"  - Variables avec codes négatifs : {(ordinal_df['has_negative_codes'] > 0).sum()}")
    print(f"  - Variables avec codes 97-99 : {(ordinal_df['has_high_codes'] > 0).sum()}")


📊 STATISTIQUES VARIABLES ORDINALES
Nombre total : 95

Missing values :
  - Moyenne : 58.68%
  - Médiane : 64.92%
  - Max : 69.91%

Cardinalité :
  - Moyenne : 27.1
  - Médiane : 22.0
  - Max : 148

Codes spéciaux :
  - Variables avec codes négatifs : 0
  - Variables avec codes 97-99 : 0


## 5. Analyse des variables catégorielles

Pour chaque variable catégorielle :
- Cardinalité (faible < 10, moyenne 10-50, haute > 50)
- % de valeurs manquantes
- Catégories rares (< 1%)
- Déséquilibre (imbalance ratio)

In [36]:
# Fonction pour analyser une variable catégorielle avec Polars
def analyze_categorical_variable(df: pl.DataFrame, var_name: str) -> dict:
    """Analyse une variable catégorielle avec Polars."""
    if var_name not in df.columns:
        return None
    
    col = df[var_name]
    
    # Statistiques de base
    cardinality = col.n_unique()
    missing_count = col.null_count()
    missing_pct = (missing_count / df.height) * 100
    
    # Value counts
    value_counts = df.group_by(var_name).agg(pl.count().alias('count')).sort('count', descending=True)
    
    # Catégories rares (< 1%)
    total = df.height - missing_count
    rare_threshold = 0.01 * total
    rare_categories = (value_counts.filter(pl.col('count') < rare_threshold)).height
    
    # Imbalance ratio
    if value_counts.height > 0:
        max_count = value_counts['count'][0]
        min_count = value_counts['count'][-1]
        imbalance_ratio = float(max_count / min_count) if min_count > 0 else float('inf')
    else:
        imbalance_ratio = 0.0
    
    # Top 5 categories - convertir en string pour éviter problèmes avec None/null
    top_5 = value_counts.head(5)
    top_5_dict = str({str(row[0]): row[1] for row in top_5.iter_rows()})
    
    # Type de cardinalité
    if cardinality < 10:
        card_type = 'faible'
    elif cardinality <= 50:
        card_type = 'moyenne'
    else:
        card_type = 'haute'
    
    analysis = {
        'variable': var_name,
        'cardinality': int(cardinality),
        'missing_count': int(missing_count),
        'missing_pct': float(missing_pct),
        'dtype': str(col.dtype),
        'rare_categories': int(rare_categories),
        'imbalance_ratio': float(imbalance_ratio),
        'top_5_categories': top_5_dict,
        'cardinality_type': card_type
    }
    
    return analysis

print("✅ Fonction analyze_categorical_variable définie")

✅ Fonction analyze_categorical_variable définie


In [37]:
# Analyser toutes les variables catégorielles
if 'categorical_vars' in locals() and len(categorical_vars) > 0:
    categorical_analysis = []
    
    print(f"Analyse de {len(categorical_vars)} variables catégorielles...")
    for var in categorical_vars:
        result = analyze_categorical_variable(df, var)
        if result:
            categorical_analysis.append(result)
    
    # Convertir en DataFrame Polars
    categorical_df = pl.DataFrame(categorical_analysis)
    print(f"\n✅ Analyse de {len(categorical_df)} variables catégorielles terminée")
    categorical_df.head(10)
else:
    print("⚠️ Liste des variables catégorielles non disponible")

Analyse de 70 variables catégorielles...

✅ Analyse de 70 variables catégorielles terminée


In [38]:
# Statistiques sur les variables catégorielles
if 'categorical_df' in locals():
    print("\n📊 STATISTIQUES VARIABLES CATÉGORIELLES")
    print("=" * 60)
    print(f"Nombre total : {categorical_df.height}")
    print(f"\nMissing values :")
    print(f"  - Moyenne : {categorical_df['missing_pct'].mean():.2f}%")
    print(f"  - Médiane : {categorical_df['missing_pct'].median():.2f}%")
    print(f"  - Max : {categorical_df['missing_pct'].max():.2f}%")
    print(f"\nCardinalité :")
    print(f"  - Moyenne : {categorical_df['cardinality'].mean():.1f}")
    print(f"  - Médiane : {categorical_df['cardinality'].median():.1f}")
    print(f"  - Max : {categorical_df['cardinality'].max()}")
    print(f"\nDistribution par type de cardinalité :")
    card_dist = categorical_df.group_by('cardinality_type').agg(pl.count().alias('count'))
    print(card_dist)
    print(f"\nCatégories rares :")
    print(f"  - Variables avec catégories rares : {(categorical_df['rare_categories'] > 0).sum()}")
    print(f"  - Moyenne catégories rares par variable : {categorical_df['rare_categories'].mean():.1f}")


📊 STATISTIQUES VARIABLES CATÉGORIELLES
Nombre total : 70

Missing values :
  - Moyenne : 47.93%
  - Médiane : 64.92%
  - Max : 83.61%

Cardinalité :
  - Moyenne : 48.7
  - Médiane : 11.0
  - Max : 707

Distribution par type de cardinalité :
shape: (3, 2)
┌──────────────────┬───────┐
│ cardinality_type ┆ count │
│ ---              ┆ ---   │
│ str              ┆ u32   │
╞══════════════════╪═══════╡
│ moyenne          ┆ 28    │
│ faible           ┆ 31    │
│ haute            ┆ 11    │
└──────────────────┴───────┘

Catégories rares :
  - Variables avec catégories rares : 39
  - Moyenne catégories rares par variable : 39.0


## 6. Focus : Variables à haute cardinalité

Analyse approfondie de STRATUM, OCOD, CNT

In [39]:
# Analyser STRATUM (1316 strates attendues)
if 'STRATUM' in df.columns:
    print("=" * 60)
    print("ANALYSE DE STRATUM")
    print("=" * 60)
    
    stratum_analysis = analyze_categorical_variable(df, 'STRATUM')
    print(f"\nCardinalité : {stratum_analysis['cardinality']}")
    print(f"Missing : {stratum_analysis['missing_pct']:.2f}%")
    print(f"Catégories rares : {stratum_analysis['rare_categories']}")
    
    # Échantillon de valeurs
    print("\nTop 10 valeurs STRATUM :")
    stratum_counts = df.group_by('STRATUM').agg(pl.count().alias('count')).sort('count', descending=True).head(10)
    print(stratum_counts)
else:
    print("⚠️ STRATUM non trouvé dans le dataset")

ANALYSE DE STRATUM

Cardinalité : 2472
Missing : 0.00%
Catégories rares : 2469

Top 10 valeurs STRATUM :
shape: (10, 2)
┌─────────┬───────┐
│ STRATUM ┆ count │
│ ---     ┆ ---   │
│ str     ┆ u32   │
╞═════════╪═══════╡
│ CAN99   ┆ 655   │
│ ITA9797 ┆ 643   │
│ THA9797 ┆ 500   │
│ AUT9797 ┆ 397   │
│ ISR9797 ┆ 396   │
│ DEU9797 ┆ 368   │
│ QCI9797 ┆ 330   │
│ NOR0001 ┆ 320   │
│ ITA97   ┆ 308   │
│ GRC0003 ┆ 278   │
└─────────┴───────┘


In [40]:
# Analyser OCOD (620 professions attendues)
# Note: OCOD peut avoir plusieurs variantes (OCOD1, OCOD2, OCOD3)
ocod_vars = [col for col in df.columns if 'OCOD' in col]
print(f"Variables OCOD trouvées : {ocod_vars}")

for ocod_var in ocod_vars:
    print("\n" + "=" * 60)
    print(f"ANALYSE DE {ocod_var} (Professions)")
    print("=" * 60)
    
    ocod_analysis = analyze_categorical_variable(df, ocod_var)
    print(f"\nCardinalité : {ocod_analysis['cardinality']}")
    print(f"Missing : {ocod_analysis['missing_pct']:.2f}%")
    
    print("\nTop 10 professions :")
    ocod_counts = df.group_by(ocod_var).agg(pl.count().alias('count')).sort('count', descending=True).head(10)
    print(ocod_counts)

Variables OCOD trouvées : ['OCOD1', 'OCOD2', 'OCOD3']

ANALYSE DE OCOD1 (Professions)

Cardinalité : 665
Missing : 30.09%

Top 10 professions :
shape: (10, 2)
┌────────┬───────┐
│ OCOD1  ┆ count │
│ ---    ┆ ---   │
│ f64    ┆ u32   │
╞════════╪═══════╡
│ null   ┆ 15047 │
│ 9701.0 ┆ 5064  │
│ 9999.0 ┆ 3503  │
│ 9705.0 ┆ 1009  │
│ 5223.0 ┆ 829   │
│ 2330.0 ┆ 745   │
│ 5120.0 ┆ 745   │
│ 2221.0 ┆ 712   │
│ 2341.0 ┆ 704   │
│ 9703.0 ┆ 626   │
└────────┴───────┘

ANALYSE DE OCOD2 (Professions)

Cardinalité : 707
Missing : 30.09%

Top 10 professions :
shape: (10, 2)
┌────────┬───────┐
│ OCOD2  ┆ count │
│ ---    ┆ ---   │
│ f64    ┆ u32   │
╞════════╪═══════╡
│ null   ┆ 15047 │
│ 9999.0 ┆ 4317  │
│ 9705.0 ┆ 1569  │
│ 8332.0 ┆ 757   │
│ 8322.0 ┆ 725   │
│ 9703.0 ┆ 583   │
│ 7111.0 ┆ 576   │
│ 9704.0 ┆ 553   │
│ 7231.0 ┆ 535   │
│ 9998.0 ┆ 466   │
└────────┴───────┘

ANALYSE DE OCOD3 (Professions)

Cardinalité : 589
Missing : 30.09%

Top 10 professions :
shape: (10, 2)
┌────────┬───────┐
│ OC

In [41]:
# Analyser CNT vs CNTRYID (redondance potentielle)
if 'CNT' in df.columns and 'CNTRYID' in df.columns:
    print("=" * 60)
    print("ANALYSE CNT vs CNTRYID (Redondance)")
    print("=" * 60)
    
    cnt_analysis = analyze_categorical_variable(df, 'CNT')
    cntryid_analysis = analyze_categorical_variable(df, 'CNTRYID')
    
    print(f"\nCNT - Cardinalité : {cnt_analysis['cardinality']}")
    print(f"CNTRYID - Cardinalité : {cntryid_analysis['cardinality']}")
    
    # Vérifier la redondance
    mapping = df.select(['CNT', 'CNTRYID']).unique()
    print(f"\nMappings uniques CNT <-> CNTRYID : {mapping.height}")
    print(f"Redondance parfaite : {mapping.height == cnt_analysis['cardinality']}")
    
    print("\nExemples de mapping :")
    print(mapping.head(10))
else:
    print("⚠️ CNT ou CNTRYID non trouvé dans le dataset")

ANALYSE CNT vs CNTRYID (Redondance)

CNT - Cardinalité : 98
CNTRYID - Cardinalité : 99

Mappings uniques CNT <-> CNTRYID : 99
Redondance parfaite : False

Exemples de mapping :
shape: (10, 2)
┌─────┬─────────┐
│ CNT ┆ CNTRYID │
│ --- ┆ ---     │
│ str ┆ f64     │
╞═════╪═════════╡
│ SLV ┆ 222.0   │
│ ISR ┆ 376.0   │
│ DEU ┆ 276.0   │
│ PRT ┆ 620.0   │
│ BGR ┆ 100.0   │
│ CRI ┆ 188.0   │
│ NLD ┆ 528.0   │
│ TUR ┆ 792.0   │
│ ARG ┆ 32.0    │
│ KAZ ┆ 398.0   │
└─────┴─────────┘


## 7. Détection de types d'échelles ordinales

Identifier :
- Échelles de Likert (3, 4, 5, 7 points)
- Échelles de fréquence
- Échelles de quantité

In [42]:
# Fonction pour détecter le type d'échelle
def detect_scale_type(unique_values, n_unique):
    """Détecte le type d'échelle ordinale."""
    
    # Échelles de Likert
    if n_unique in [3, 4, 5, 7]:
        # Vérifier si les valeurs sont consécutives (ex: 1,2,3,4,5)
        if all(isinstance(v, (int, np.integer)) for v in unique_values):
            sorted_vals = sorted(unique_values)
            if sorted_vals == list(range(min(sorted_vals), max(sorted_vals) + 1)):
                return f'likert_{n_unique}'
    
    # Échelles avec valeurs négatives (codes spéciaux)
    if any(v < 0 for v in unique_values if isinstance(v, (int, float, np.number))):
        return 'with_special_codes'
    
    # Échelles de quantité par ranges
    if n_unique >= 5 and n_unique <= 10:
        return 'quantity_ranges'
    
    return 'other'

print("✅ Fonction detect_scale_type définie")

✅ Fonction detect_scale_type définie


In [43]:
# Détecter types d'échelles pour les variables ordinales
if 'ordinal_df' in locals():
    # Ajouter colonne scale_type
    scale_types = []
    for row in ordinal_df.iter_rows(named=True):
        scale_type = detect_scale_type(row['unique_values'], row['n_unique'])
        scale_types.append(scale_type)
    
    ordinal_df = ordinal_df.with_columns(pl.Series('scale_type', scale_types))
    
    print("\nDistribution des types d'échelles ordinales :")
    scale_dist = ordinal_df.group_by('scale_type').agg(pl.count().alias('count'))
    print(scale_dist)
    
    print("\nExemples par type d'échelle :")
    for scale_type in ordinal_df['scale_type'].unique().to_list():
        print(f"\n{scale_type.upper()} :")
        examples = ordinal_df.filter(pl.col('scale_type') == scale_type).select(['variable', 'n_unique', 'unique_values']).head(3)
        print(examples)


Distribution des types d'échelles ordinales :
shape: (2, 2)
┌─────────────────┬───────┐
│ scale_type      ┆ count │
│ ---             ┆ ---   │
│ str             ┆ u32   │
╞═════════════════╪═══════╡
│ other           ┆ 76    │
│ quantity_ranges ┆ 19    │
└─────────────────┴───────┘

Exemples par type d'échelle :

QUANTITY_RANGES :
shape: (3, 3)
┌──────────┬──────────┬─────────────────────────────────┐
│ variable ┆ n_unique ┆ unique_values                   │
│ ---      ┆ ---      ┆ ---                             │
│ str      ┆ i64      ┆ str                             │
╞══════════╪══════════╪═════════════════════════════════╡
│ ISCEDP   ┆ 8        ┆ [244.0, 254.0, 341.0, 343.0, 3… │
│ ST253    ┆ 9        ┆ [0.0, 0.1428571428571428, 0.28… │
│ PA195    ┆ 8        ┆ [0.0, 0.1666666666666666, 0.33… │
└──────────┴──────────┴─────────────────────────────────┘

OTHER :
shape: (3, 3)
┌──────────┬──────────┬─────────────────────────────────┐
│ variable ┆ n_unique ┆ unique_values           

## 8. Analyse des valeurs manquantes

Patterns de missing values

In [44]:
# Statistiques globales sur les valeurs manquantes avec Polars
missing_stats = pl.DataFrame({
    'variable': df.columns,
    'missing_count': [df[col].null_count() for col in df.columns],
    'missing_pct': [(df[col].null_count() / df.height) * 100 for col in df.columns]
}).sort('missing_pct', descending=True)

print("Top 20 variables avec le plus de valeurs manquantes :")
missing_stats.head(20)

Top 20 variables avec le plus de valeurs manquantes :


variable,missing_count,missing_pct
str,i64,f64
"""reading_q13_average_score""",50000,100.0
"""reading_q14_average_score""",50000,100.0
"""reading_q15_average_score""",50000,100.0
"""math_q17_average_score""",50000,100.0
"""math_q18_average_score""",50000,100.0
…,…,…
"""reading_q13_total_timing""",50000,100.0
"""reading_q14_total_timing""",50000,100.0
"""reading_q15_total_timing""",50000,100.0


## 9. Résumé et conclusions préliminaires

In [45]:
# Résumé des découvertes
print("=" * 80)
print("RÉSUMÉ DE L'ANALYSE EXPLORATOIRE")
print("=" * 80)

print(f"\n📊 DATASET")
print(f"  - Taille de l'échantillon : {df.height:,} lignes")
print(f"  - Nombre de colonnes : {df.width}")
print(f"  - Mémoire : {df.estimated_size('mb'):.2f} MB")

if 'ordinal_df' in locals():
    print(f"\n📈 VARIABLES ORDINALES")
    print(f"  - Nombre : {ordinal_df.height}")
    print(f"  - Missing moyen : {ordinal_df['missing_pct'].mean():.2f}%")
    print(f"  - Cardinalité moyenne : {ordinal_df['n_unique'].mean():.1f}")

if 'categorical_df' in locals():
    print(f"\n🏷️  VARIABLES CATÉGORIELLES")
    print(f"  - Nombre : {categorical_df.height}")
    print(f"  - Missing moyen : {categorical_df['missing_pct'].mean():.2f}%")
    print(f"  - Cardinalité moyenne : {categorical_df['cardinality'].mean():.1f}")
    print(f"  - Haute cardinalité (>50) : {(categorical_df['cardinality'] > 50).sum()}")

print(f"\n⚠️  VARIABLES À FORTE MISSING (>50%)")
high_missing = missing_stats.filter(pl.col('missing_pct') > 50)
print(f"  - Nombre : {high_missing.height}")
print(f"  - À considérer pour suppression")

print("\n" + "=" * 80)

RÉSUMÉ DE L'ANALYSE EXPLORATOIRE

📊 DATASET
  - Taille de l'échantillon : 50,000 lignes
  - Nombre de colonnes : 307
  - Mémoire : 102.50 MB

📈 VARIABLES ORDINALES
  - Nombre : 95
  - Missing moyen : 58.68%
  - Cardinalité moyenne : 27.1

🏷️  VARIABLES CATÉGORIELLES
  - Nombre : 70
  - Missing moyen : 47.93%
  - Cardinalité moyenne : 48.7
  - Haute cardinalité (>50) : 11

⚠️  VARIABLES À FORTE MISSING (>50%)
  - Nombre : 207
  - À considérer pour suppression



## 10. Détection de variables redondantes / corrélées

Objectifs :
- Calculer corrélations de Spearman pour paires de variables ordinales
- Calculer Cramér's V pour paires de variables catégorielles  
- Identifier variables dupliquées
- **Output** : Liste de variables à supprimer

In [46]:
# 10.1 - Fonction pour calculer Cramér's V entre deux variables catégorielles
def cramers_v(df: pl.DataFrame, var1: str, var2: str) -> float:
    """Calcule le coefficient de Cramér's V entre deux variables catégorielles."""
    # Créer une table de contingence avec Polars
    contingency = df.group_by([var1, var2]).agg(pl.count().alias('count'))
    
    # Convertir en pandas pour utiliser chi2_contingency
    contingency_pd = contingency.to_pandas()
    ct = pd.crosstab(
        df[var1].to_pandas(), 
        df[var2].to_pandas()
    )
    
    # Calculer chi2
    chi2, _, _, _ = chi2_contingency(ct)
    n = ct.sum().sum()
    
    # Cramér's V
    min_dim = min(ct.shape[0] - 1, ct.shape[1] - 1)
    if min_dim == 0:
        return 0.0
    
    cramers = np.sqrt(chi2 / (n * min_dim))
    return cramers

print("✅ Fonction cramers_v définie")

✅ Fonction cramers_v définie


In [47]:
# 10.2 - Détecter variables dupliquées (ST006, ST008, PA008)
# Selon glossaire_analysis.md, il y a des codes dupliqués identifiés

duplicated_candidates = [col for col in df.columns if 'duplicated' in col.lower()]
print(f"Variables avec '_duplicated' dans le nom : {duplicated_candidates}")

# Vérifier les variables ST006, ST008, PA008 et leurs duplicatas
vars_to_check = {
    'ST006': ['ST006', 'ST006_duplicated_0'],
    'ST008': ['ST008', 'ST008_duplicated_0'],
    'PA008': ['PA008', 'PA008_duplicated_0']
}

redundant_vars = []

for original, versions in vars_to_check.items():
    available = [v for v in versions if v in df.columns]
    
    if len(available) == 2:
        v1, v2 = available
        
        # Calculer corrélation de Spearman (pour variables ordinales/numériques)
        # Convertir en pandas temporairement pour scipy
        df_temp = df.select([v1, v2]).to_pandas()
        
        # Retirer les valeurs manquantes
        df_temp_clean = df_temp.dropna()
        
        if len(df_temp_clean) > 0:
            corr, pval = spearmanr(df_temp_clean[v1], df_temp_clean[v2])
            
            print(f"\n{original}:")
            print(f"  - Corrélation Spearman : {corr:.4f} (p-value: {pval:.4e})")
            print(f"  - Valeurs identiques : {(df[v1] == df[v2]).sum()} / {df.height}")
            
            # Si corrélation très forte (>0.95), marquer comme redondant
            if abs(corr) > 0.95:
                redundant_vars.append(v2)  # Garder l'original, supprimer le duplicata
                print(f"  ➡️ REDONDANT : à supprimer {v2}")

print(f"\n✅ Variables redondantes identifiées : {redundant_vars}")

Variables avec '_duplicated' dans le nom : ['ST008_duplicated_0', 'ST006_duplicated_0', 'PA008_duplicated_0']

✅ Variables redondantes identifiées : []


In [48]:
# 10.3 - Vérifier redondance CNT vs CNTRYID (déjà identifié en section 6)
# Ces deux variables encodent la même information (pays)

print("=" * 60)
print("ANALYSE REDONDANCE CNT vs CNTRYID")
print("=" * 60)

# Vérifier mapping 1-to-1
mapping = df.select(['CNT', 'CNTRYID']).unique()
cnt_unique = df['CNT'].n_unique()
cntryid_unique = df['CNTRYID'].n_unique()

print(f"\nCNT cardinalité : {cnt_unique}")
print(f"CNTRYID cardinalité : {cntryid_unique}")
print(f"Mappings uniques : {mapping.height}")

# Vérifier si chaque CNT correspond à un seul CNTRYID
cnt_to_cntryid = df.group_by('CNT').agg(pl.col('CNTRYID').n_unique().alias('n_cntryid'))
perfect_mapping = (cnt_to_cntryid['n_cntryid'] == 1).all()

print(f"\nMapping 1-to-1 parfait : {perfect_mapping}")

if perfect_mapping:
    print("➡️ REDONDANCE CONFIRMÉE : CNT et CNTRYID encodent la même information")
    print("   Recommandation : garder CNT (plus lisible) et supprimer CNTRYID")
    redundant_vars.append('CNTRYID')
else:
    print("⚠️ Mapping imparfait détecté - vérifier manuellement")

print(f"\n✅ Total variables redondantes : {len(redundant_vars)}")

ANALYSE REDONDANCE CNT vs CNTRYID

CNT cardinalité : 98
CNTRYID cardinalité : 99
Mappings uniques : 99

Mapping 1-to-1 parfait : False
⚠️ Mapping imparfait détecté - vérifier manuellement

✅ Total variables redondantes : 0


In [49]:
# 10.4 - Chercher corrélations fortes entre variables ordinales (échantillon)
# Note: Calculer toutes les paires serait coûteux (95*94/2 = 4465 paires)
# On se concentre sur un échantillon ou les variables avec cardinalité modérée

print("=" * 60)
print("CORRÉLATIONS ENTRE VARIABLES ORDINALES (échantillon)")
print("=" * 60)

# Sélectionner variables ordinales avec cardinalité modérée (5-50) et pas trop de missing
ordinal_moderate = ordinal_df.filter(
    (pl.col('n_unique') >= 5) & 
    (pl.col('n_unique') <= 50) &
    (pl.col('missing_pct') < 70)
)['variable'].to_list()

print(f"\nVariables ordinales avec cardinalité modérée et <70% missing : {len(ordinal_moderate)}")
print(f"Exemples : {ordinal_moderate[:10]}")

# Calculer matrice de corrélation pour un sous-ensemble (10 premières variables)
if len(ordinal_moderate) > 0:
    sample_vars = ordinal_moderate[:min(10, len(ordinal_moderate))]
    
    # Convertir en pandas pour corrélation
    df_sample = df.select(sample_vars).to_pandas()
    
    # Calculer corrélation de Spearman
    corr_matrix = df_sample.corr(method='spearman')
    
    print(f"\nMatrice de corrélation (Spearman) pour {len(sample_vars)} variables :")
    print(corr_matrix)
    
    # Identifier paires avec forte corrélation (>0.8, hors diagonale)
    strong_corr = []
    for i, var1 in enumerate(sample_vars):
        for j, var2 in enumerate(sample_vars):
            if i < j:  # Éviter doublons et diagonale
                corr_val = corr_matrix.loc[var1, var2]
                if abs(corr_val) > 0.8 and not np.isnan(corr_val):
                    strong_corr.append((var1, var2, corr_val))
    
    if strong_corr:
        print(f"\n⚠️ Paires fortement corrélées (|r| > 0.8) :")
        for v1, v2, corr in strong_corr:
            print(f"  - {v1} <-> {v2} : {corr:.3f}")
    else:
        print("\n✅ Pas de corrélations fortes détectées dans cet échantillon")

print("\nNote: Analyse complète nécessiterait calcul de toutes les paires")

CORRÉLATIONS ENTRE VARIABLES ORDINALES (échantillon)

Variables ordinales avec cardinalité modérée et <70% missing : 85
Exemples : ['ISCEDP', 'ST253', 'ST311', 'ST038', 'IC180', 'ST255', 'ST034', 'PA195', 'ST230', 'ST354']

Matrice de corrélation (Spearman) pour 10 variables :
          ISCEDP     ST253     ST311     ST038     IC180     ST255     ST034  \
ISCEDP  1.000000  0.045147  0.080035 -0.064641  0.016340  0.021478 -0.017170   
ST253   0.045147  1.000000  0.164897  0.024930  0.191845  0.438111  0.031810   
ST311   0.080035  0.164897  1.000000  0.093522  0.189675  0.145026  0.132295   
ST038  -0.064641  0.024930  0.093522  1.000000 -0.005478  0.025445  0.196612   
IC180   0.016340  0.191845  0.189675 -0.005478  1.000000  0.185604  0.026033   
ST255   0.021478  0.438111  0.145026  0.025445  0.185604  1.000000  0.027535   
ST034  -0.017170  0.031810  0.132295  0.196612  0.026033  0.027535  1.000000   
PA195   0.115385  0.015889  0.060740 -0.050178  0.114866  0.047120 -0.019596   
ST

## 11. Analyse des patterns de valeurs manquantes

Objectifs :
- Identifier si les valeurs manquantes sont MCAR (Missing Completely At Random), MAR (Missing At Random), ou MNAR (Missing Not At Random)
- Détecter corrélations entre valeurs manquantes de différentes variables
- Décider stratégie d'imputation par variable
- Identifier variables à supprimer (trop de missing)

In [51]:
# 11.1 - Créer matrice binaire des valeurs manquantes
print("=" * 60)
print("MATRICE DE VALEURS MANQUANTES")
print("=" * 60)

# Créer une matrice binaire : 1 si manquant, 0 sinon
# On se concentre sur variables ordinales et catégorielles (pas toutes les 307 colonnes)
vars_of_interest = ordinal_vars + categorical_vars

missing_matrix = pl.DataFrame({
    col: df[col].is_null().cast(pl.Int8)
    for col in vars_of_interest
})

print(f"\nMatrice créée : {missing_matrix.shape}")
print(f"Variables analysées : {len(vars_of_interest)}")

# Statistiques de base
total_cells = missing_matrix.height * missing_matrix.width
total_missing = sum([missing_matrix[col].sum() for col in missing_matrix.columns])
missing_pct_overall = (total_missing / total_cells) * 100

print(f"\n📊 Statistiques globales :")
print(f"  - Total cellules : {total_cells:,}")
print(f"  - Cellules manquantes : {total_missing:,}")
print(f"  - Pourcentage global : {missing_pct_overall:.2f}%")

MATRICE DE VALEURS MANQUANTES

Matrice créée : (50000, 162)
Variables analysées : 165

📊 Statistiques globales :
  - Total cellules : 8,100,000
  - Cellules manquantes : 4,367,177
  - Pourcentage global : 53.92%


In [52]:
# 11.2 - Identifier variables avec trop de missing (>80%)
# Ces variables sont candidates à la suppression

high_missing_vars = missing_stats.filter(pl.col('missing_pct') > 80)['variable'].to_list()

print("=" * 60)
print("VARIABLES AVEC >80% DE VALEURS MANQUANTES")
print("=" * 60)

print(f"\nNombre total : {len(high_missing_vars)}")
print(f"\nPremières 20 variables :")
for var in high_missing_vars[:20]:
    missing_pct = missing_stats.filter(pl.col('variable') == var)['missing_pct'][0]
    print(f"  - {var}: {missing_pct:.2f}%")

# Vérifier si ce sont des variables ordinales/catégorielles
high_missing_ordinal = [v for v in high_missing_vars if v in ordinal_vars]
high_missing_categorical = [v for v in high_missing_vars if v in categorical_vars]

print(f"\nDont ordinales : {len(high_missing_ordinal)}")
print(f"Dont catégorielles : {len(high_missing_categorical)}")

print("\n⚠️ Recommandation : Supprimer ces variables (trop peu d'information)")

VARIABLES AVEC >80% DE VALEURS MANQUANTES

Nombre total : 51

Premières 20 variables :
  - reading_q13_average_score: 100.00%
  - reading_q14_average_score: 100.00%
  - reading_q15_average_score: 100.00%
  - math_q17_average_score: 100.00%
  - math_q18_average_score: 100.00%
  - math_q19_average_score: 100.00%
  - math_q20_average_score: 100.00%
  - math_q21_average_score: 100.00%
  - science_q13_average_score: 100.00%
  - science_q14_average_score: 100.00%
  - science_q15_average_score: 100.00%
  - science_q16_average_score: 100.00%
  - science_q17_average_score: 100.00%
  - science_q18_average_score: 100.00%
  - science_q19_average_score: 100.00%
  - reading_q13_total_timing: 100.00%
  - reading_q14_total_timing: 100.00%
  - reading_q15_total_timing: 100.00%
  - math_q17_total_timing: 100.00%
  - math_q18_total_timing: 100.00%

Dont ordinales : 0
Dont catégorielles : 1

⚠️ Recommandation : Supprimer ces variables (trop peu d'information)


In [56]:
high_missing_ordinal

[]

In [57]:
high_missing_categorical

['LANGTEST_PAQ']

A supprimer: LANGTEST_PAQ (trop de NaN)

In [53]:
# 11.3 - Calculer corrélations entre patterns de missing (échantillon)
# Pour détecter si certaines variables manquent ensemble (indicateur de MAR/MNAR)

print("=" * 60)
print("CORRÉLATIONS ENTRE PATTERNS DE MISSING")
print("=" * 60)

# Sélectionner variables avec missing modéré (20-80%) pour analyse
moderate_missing_vars = missing_stats.filter(
    (pl.col('missing_pct') >= 20) & 
    (pl.col('missing_pct') <= 80)
)['variable'].to_list()

# Garder seulement variables ordinales/catégorielles
moderate_missing_vars = [v for v in moderate_missing_vars if v in vars_of_interest]

print(f"\nVariables avec missing modéré (20-80%) : {len(moderate_missing_vars)}")

# Prendre un échantillon de 15 variables max pour la matrice de corrélation
sample_missing_vars = moderate_missing_vars[:min(15, len(moderate_missing_vars))]

if len(sample_missing_vars) > 1:
    # Créer sous-matrice de missing pour ces variables
    missing_sample = missing_matrix.select(sample_missing_vars).to_pandas()
    
    # Calculer corrélation entre patterns de missing
    missing_corr = missing_sample.corr(method='pearson')
    
    print(f"\nMatrice de corrélation des patterns de missing ({len(sample_missing_vars)} variables) :")
    print(missing_corr.round(3))
    
    # Identifier paires fortement corrélées
    strong_missing_corr = []
    for i, var1 in enumerate(sample_missing_vars):
        for j, var2 in enumerate(sample_missing_vars):
            if i < j:
                corr_val = missing_corr.loc[var1, var2]
                if abs(corr_val) > 0.5:
                    strong_missing_corr.append((var1, var2, corr_val))
    
    if strong_missing_corr:
        print(f"\n⚠️ Paires avec patterns de missing corrélés (|r| > 0.5) :")
        for v1, v2, corr in strong_missing_corr[:10]:  # Top 10
            print(f"  - {v1} <-> {v2} : {corr:.3f}")
        print("\n➡️ Cela suggère MAR/MNAR : certaines variables manquent ensemble")
    else:
        print("\n✅ Patterns de missing indépendants (suggère MCAR)")
else:
    print("\n⚠️ Pas assez de variables avec missing modéré pour analyser les corrélations")

CORRÉLATIONS ENTRE PATTERNS DE MISSING

Variables avec missing modéré (20-80%) : 150

Matrice de corrélation des patterns de missing (15 variables) :
          MATHEASE  MISSSC  ST095  PA033  ST146  ST100  ST097  ST113  PA032  \
MATHEASE     1.000   0.803 -0.418 -0.418 -0.418 -0.418 -0.418 -0.418 -0.418   
MISSSC       0.803   1.000 -0.428 -0.428 -0.428 -0.428 -0.428 -0.428 -0.428   
ST095       -0.418  -0.428  1.000  1.000  1.000  1.000  1.000  1.000  1.000   
PA033       -0.418  -0.428  1.000  1.000  1.000  1.000  1.000  1.000  1.000   
ST146       -0.418  -0.428  1.000  1.000  1.000  1.000  1.000  1.000  1.000   
ST100       -0.418  -0.428  1.000  1.000  1.000  1.000  1.000  1.000  1.000   
ST097       -0.418  -0.428  1.000  1.000  1.000  1.000  1.000  1.000  1.000   
ST113       -0.418  -0.428  1.000  1.000  1.000  1.000  1.000  1.000  1.000   
PA032       -0.418  -0.428  1.000  1.000  1.000  1.000  1.000  1.000  1.000   
IC001       -0.418  -0.428  1.000  1.000  1.000  1.000  1.00

In [54]:
# 11.4 - Stratégie d'imputation recommandée par variable
print("=" * 80)
print("STRATÉGIES D'IMPUTATION RECOMMANDÉES")
print("=" * 80)

# Catégoriser les variables par niveau de missing et proposer stratégies
imputation_strategies = {
    'supprimer (>80% missing)': [],
    'imputation complexe (50-80% missing)': [],
    'imputation simple (20-50% missing)': [],
    'imputation minimale (<20% missing)': [],
    'pas de missing': []
}

for var in vars_of_interest:
    missing_pct = missing_stats.filter(pl.col('variable') == var)['missing_pct'][0]
    
    if missing_pct > 80:
        imputation_strategies['supprimer (>80% missing)'].append(var)
    elif missing_pct > 50:
        imputation_strategies['imputation complexe (50-80% missing)'].append(var)
    elif missing_pct > 20:
        imputation_strategies['imputation simple (20-50% missing)'].append(var)
    elif missing_pct > 0:
        imputation_strategies['imputation minimale (<20% missing)'].append(var)
    else:
        imputation_strategies['pas de missing'].append(var)

print("\n📊 Distribution des variables par stratégie :\n")
for strategy, vars_list in imputation_strategies.items():
    print(f"{strategy}: {len(vars_list)} variables")

# Recommandations détaillées
print("\n" + "=" * 80)
print("RECOMMANDATIONS DÉTAILLÉES")
print("=" * 80)

print(f"\n1️⃣ SUPPRIMER ({len(imputation_strategies['supprimer (>80% missing)'])} variables)")
print("   Trop peu d'information pour être utiles")
if imputation_strategies['supprimer (>80% missing)']:
    print(f"   Exemples : {imputation_strategies['supprimer (>80% missing)'][:5]}")

print(f"\n2️⃣ IMPUTATION COMPLEXE ({len(imputation_strategies['imputation complexe (50-80% missing)'])} variables)")
print("   Options : KNN imputer, MICE, ou créer flag 'is_missing'")
if imputation_strategies['imputation complexe (50-80% missing)']:
    print(f"   Exemples : {imputation_strategies['imputation complexe (50-80% missing)'][:5]}")

print(f"\n3️⃣ IMPUTATION SIMPLE ({len(imputation_strategies['imputation simple (20-50% missing)'])} variables)")
print("   Ordinales : médiane ou mode")
print("   Catégorielles : mode ou catégorie 'Unknown'")
if imputation_strategies['imputation simple (20-50% missing)']:
    print(f"   Exemples : {imputation_strategies['imputation simple (20-50% missing)'][:5]}")

print(f"\n4️⃣ IMPUTATION MINIMALE ({len(imputation_strategies['imputation minimale (<20% missing)'])} variables)")
print("   Mode pour catégorielles, médiane pour ordinales")
if imputation_strategies['imputation minimale (<20% missing)']:
    print(f"   Exemples : {imputation_strategies['imputation minimale (<20% missing)'][:5]}")

print(f"\n5️⃣ PAS DE MISSING ({len(imputation_strategies['pas de missing'])} variables)")
print("   Aucune imputation nécessaire")
if imputation_strategies['pas de missing']:
    print(f"   Exemples : {imputation_strategies['pas de missing'][:5]}")

STRATÉGIES D'IMPUTATION RECOMMANDÉES

📊 Distribution des variables par stratégie :

supprimer (>80% missing): 1 variables
imputation complexe (50-80% missing): 118 variables
imputation simple (20-50% missing): 35 variables
imputation minimale (<20% missing): 4 variables
pas de missing: 7 variables

RECOMMANDATIONS DÉTAILLÉES

1️⃣ SUPPRIMER (1 variables)
   Trop peu d'information pour être utiles
   Exemples : ['LANGTEST_PAQ']

2️⃣ IMPUTATION COMPLEXE (118 variables)
   Options : KNN imputer, MICE, ou créer flag 'is_missing'
   Exemples : ['ISCEDP', 'ST253', 'ST311', 'IC180', 'PA195']

3️⃣ IMPUTATION SIMPLE (35 variables)
   Ordinales : médiane ou mode
   Catégorielles : mode ou catégorie 'Unknown'
   Exemples : ['IC171', 'ST038', 'ST255', 'ST034', 'IC173']

4️⃣ IMPUTATION MINIMALE (4 variables)
   Mode pour catégorielles, médiane pour ordinales
   Exemples : ['LANGTEST_QQQ', 'ST003D02T', 'ST003D03T', 'ST004D01T']

5️⃣ PAS DE MISSING (7 variables)
   Aucune imputation nécessaire
   Exem

================================================================================
STRATÉGIES D'IMPUTATION RECOMMANDÉES
================================================================================

📊 Distribution des variables par stratégie :

supprimer (>80% missing): 1 variables
imputation complexe (50-80% missing): 118 variables
imputation simple (20-50% missing): 35 variables
imputation minimale (<20% missing): 4 variables
pas de missing: 7 variables

================================================================================
RECOMMANDATIONS DÉTAILLÉES
================================================================================

1️⃣ SUPPRIMER (1 variables)
   Trop peu d'information pour être utiles
   Exemples : ['LANGTEST_PAQ']

2️⃣ IMPUTATION COMPLEXE (118 variables)
   Options : KNN imputer
   Exemples : ['ISCEDP', 'ST253', 'ST311', 'IC180', 'PA195']

3️⃣ IMPUTATION SIMPLE (35 variables)
   Ordinales : médiane ou mode
   Catégorielles : mode ou catégorie 'Unknown'
   Exemples : ['IC171', 'ST038', 'ST255', 'ST034', 'IC173']

4️⃣ IMPUTATION MINIMALE (4 variables)
   Mode pour catégorielles, médiane pour ordinales
   Exemples : ['LANGTEST_QQQ', 'ST003D02T', 'ST003D03T', 'ST004D01T']

5️⃣ PAS DE MISSING (7 variables)
   Aucune imputation nécessaire
   Exemples : ['CYC', 'NatCen', 'SUBNATIO', 'OECD', 'ADMINMODE']

## 12. Synthèse finale et recommandations

In [ ]:
# 12. SYNTHÈSE FINALE - Toutes les recommandations pour le preprocessing

print("=" * 80)
print("SYNTHÈSE FINALE - RECOMMANDATIONS POUR LE PREPROCESSING")
print("=" * 80)

print("\n📊 STATISTIQUES GLOBALES")
print("-" * 80)
print(f"Dataset : {df.height:,} lignes × {df.width} colonnes")
print(f"Variables ordinales : {len(ordinal_vars)}")
print(f"Variables catégorielles : {len(categorical_vars)}")
print(f"Missing global : {missing_pct_overall:.2f}%")

print("\n🗑️  VARIABLES À SUPPRIMER")
print("-" * 80)

# Combiner toutes les variables à supprimer
vars_to_remove = list(set(
    vars_to_drop +  # Redondantes
    imputation_strategies['supprimer (>80% missing)']  # Trop de missing
))

print(f"Total : {len(vars_to_remove)} variables")
print(f"  - Redondantes : {len(vars_to_drop)}")
print(f"  - >80% missing : {len(imputation_strategies['supprimer (>80% missing)'])}")
print(f"\nExemples : {vars_to_remove[:10]}")

print("\n📈 VARIABLES ORDINALES - Traitement")
print("-" * 80)
print(f"Total : {len(ordinal_vars)}")
print(f"Cardinalité moyenne : {ordinal_df['n_unique'].mean():.1f}")
print(f"Missing moyen : {ordinal_df['missing_pct'].mean():.2f}%")
print("\nRecommandations :")
print("  1. Encodage ordinal (LabelEncoder) pour préserver l'ordre")
print("  2. Imputation selon niveau de missing (voir section 11)")
print("  3. Normalisation/standardisation si nécessaire")

print("\n🏷️  VARIABLES CATÉGORIELLES - Traitement")
print("-" * 80)
print(f"Total : {len(categorical_vars)}")
print(f"Cardinalité moyenne : {categorical_df['cardinality'].mean():.1f}")
print(f"Missing moyen : {categorical_df['missing_pct'].mean():.2f}%")

# Grouper par cardinalité
cat_faible = categorical_df.filter(pl.col('cardinality_type') == 'faible').height
cat_moyenne = categorical_df.filter(pl.col('cardinality_type') == 'moyenne').height
cat_haute = categorical_df.filter(pl.col('cardinality_type') == 'haute').height

print(f"\nDistribution :")
print(f"  - Faible cardinalité (<10) : {cat_faible} → One-Hot Encoding")
print(f"  - Moyenne cardinalité (10-50) : {cat_moyenne} → Target Encoding ou Frequency Encoding")
print(f"  - Haute cardinalité (>50) : {cat_haute} → Target Encoding + dimensionality reduction")

print("\n⚠️  VARIABLES À HAUTE CARDINALITÉ - Attention particulière")
print("-" * 80)
high_card_vars = categorical_df.filter(pl.col('cardinality') > 50)['variable'].to_list()
print(f"Variables concernées : {len(high_card_vars)}")
for var in high_card_vars:
    card = categorical_df.filter(pl.col('variable') == var)['cardinality'][0]
    print(f"  - {var}: {card} catégories")
print("\nRecommandations :")
print("  • STRATUM (2472 catégories) : Target encoding ou regroupement par patterns")
print("  • OCOD1/2/3 (600-700 catégories) : Target encoding ou regroupement ISCO")
print("  • Autres : Frequency encoding ou regroupement catégories rares")

print("\n💡 STRATÉGIES D'IMPUTATION")
print("-" * 80)
for strategy, vars_list in imputation_strategies.items():
    if strategy != 'supprimer (>80% missing)':
        print(f"{strategy}: {len(vars_list)} variables")

print("\n✅ PROCHAINES ÉTAPES")
print("-" * 80)
print("1. Créer classe OrdinalPreprocessor")
print("2. Créer classe CategoricalPreprocessor") 
print("3. Implémenter méthodes d'imputation adaptatives")
print("4. Implémenter encodages par type de cardinalité")
print("5. Créer pipeline complet avec validation")
print("6. Tester sur X_train complet")

print("\n" + "=" * 80)

================================================================================
SYNTHÈSE FINALE - RECOMMANDATIONS POUR LE PREPROCESSING
================================================================================

📊 STATISTIQUES GLOBALES
--------------------------------------------------------------------------------
Dataset : 50,000 lignes × 307 colonnes
Variables ordinales : 95
Variables catégorielles : 70
Missing global : 53.92%

🗑️  VARIABLES À SUPPRIMER
--------------------------------------------------------------------------------
Total : 1 variables
  - Redondantes : 0
  - >80% missing : 1

Exemples : ['LANGTEST_PAQ']

📈 VARIABLES ORDINALES - Traitement
--------------------------------------------------------------------------------
Total : 95
Cardinalité moyenne : 27.1
Missing moyen : 58.68%

Recommandations :
  1. Encodage ordinal (LabelEncoder) pour préserver l'ordre
  2. Imputation selon niveau de missing (voir section 11)
  3. Normalisation/standardisation si nécessaire

🏷️  VARIABLES CATÉGORIELLES - Traitement
--------------------------------------------------------------------------------
Total : 70
Cardinalité moyenne : 48.7
Missing moyen : 47.93%

Distribution :
  - Faible cardinalité (<10) : 31 → One-Hot Encoding
  - Moyenne cardinalité (10-50) : 28 → Target Encoding ou Frequency Encoding
  - Haute cardinalité (>50) : 11 → Target Encoding + dimensionality reduction

⚠️  VARIABLES À HAUTE CARDINALITÉ - Attention particulière
--------------------------------------------------------------------------------
Variables concernées : 11
  - NatCen: 96 catégories
  - SUBNATIO: 123 catégories
  - LANGTEST_QQQ: 61 catégories
  - LANGTEST_COG: 62 catégories
  - OCOD1: 665 catégories
  - OCOD2: 707 catégories
  - OCOD3: 589 catégories
  - ST296: 143 catégories
  - PA006: 107 catégories
  - WB162: 114 catégories
  - EC153: 74 catégories

Recommandations :
  • STRATUM (2472 catégories) : Target encoding ou regroupement par patterns
  • OCOD1/2/3 (600-700 catégories) : Target encoding ou regroupement ISCO
  • Autres : Frequency encoding ou regroupement catégories rares

💡 STRATÉGIES D'IMPUTATION
--------------------------------------------------------------------------------
imputation complexe (50-80% missing): 118 variables
imputation simple (20-50% missing): 35 variables
imputation minimale (<20% missing): 4 variables
pas de missing: 7 variables